# Download data

In [ ]:
# !curl -s https://d1bjgq97if6urz.cloudfront.net/Public/Peilingwijzer/Last/Cijfers_Peilingwijzer.xlsx -o Cijfers_Peilingwijzer.xlsx

# Install dependencies
With your favorite package manager, right from the comfort of your notebook.

In [ ]:
# %conda install pandas openpyxl matplotlib ipywidgets 
# %pip install pandas openpyxl matplotlib ipywidgets

# Import data
The Excel file when first downloaded for some reason doesn't want to load for me. If I open it in Excel and manually save it, it works again. /shrug

In [ ]:
import pandas as pd

In [ ]:
numbers = pd.read_excel('Cijfers_Peilingwijzer.xlsx', header=0, index_col=0, engine='openpyxl',
                       )

## Sanity check: recreate plot from Peilingwijzer
https://peilingwijzer.tomlouwerse.nl/p/laatste-cijfers.html

In [ ]:
zetels = numbers.drop(columns=['Datum', 'Percentage', 'PercentageLaag', 'PercentageHoog'])

In [ ]:
zetels['ZetelsHoog'] = zetels['ZetelsHoog'] - zetels['Zetels']
zetels['Zetels'] = zetels['Zetels'] - zetels['ZetelsLaag']

In [ ]:
zetels = zetels.reindex(['ZetelsLaag', 'Zetels', 'ZetelsHoog'], axis=1)

In [ ]:
zetels.plot.bar(stacked=True, figsize=(11, 8))

# Coalitiewijzer!
Press the buttons and get the number of seats of your selected coalition. A majority coalition needs 76 seats.

In [ ]:
import ipywidgets as widgets

In [ ]:
toggle_buttons = {key: widgets.ToggleButton(description=key, value=False,
                                            layout=widgets.Layout(height='auto', width='auto'))
                  for key in numbers.index}

grid_width = 5

grid = widgets.GridspecLayout((len(numbers) - 1) // grid_width + 1, grid_width)
for ix, (key, button) in enumerate(toggle_buttons.items()):
    ix1, ix2 = ix // grid_width, ix % grid_width
    grid[ix1, ix2] = button
    
balls = widgets.GridspecLayout(10, 15)

In [ ]:
def on_toggle(**toggles):
    expectation_value = numbers[list(toggles.values())].Zetels.sum()
    low = numbers[list(toggles.values())].ZetelsLaag.sum()
    high = numbers[list(toggles.values())].ZetelsHoog.sum()
    print(f"verwacht aantal zetels: {expectation_value} (laag: {low}, hoog: {high})")
    
    count = 0
    for ix in range(150):
        ix1, ix2 = ix // 15, ix % 15
        if count > high:
            value = '.'
        elif count > expectation_value:
            value = 'o'
        elif count == expectation_value:
            value = 'O'
        else:
            value = 'o'
        
        balls[ix1, ix2] = widgets.HTML(value=value, disabled=False,
                                       layout=widgets.Layout(height='auto', width='auto'),
                                       )

interact_out = widgets.interactive_output(on_toggle, toggle_buttons)
display(grid, interact_out)

In [ ]:
# display(balls)

In [ ]:
bar = widgets.IntProgress(
    value=0,
    min=0,
    max=150,
    step=1,
    description='coalition size:',
    orientation='horizontal'
)

In [ ]:
def on_toggle(**toggles):
    expectation_value = numbers[list(toggles.values())].Zetels.sum()
    low = numbers[list(toggles.values())].ZetelsLaag.sum()
    high = numbers[list(toggles.values())].ZetelsHoog.sum()
    print(f"verwacht aantal zetels: {expectation_value} (laag: {low}, hoog: {high})")
    
    bar.value = expectation_value

interact_out = widgets.interactive_output(on_toggle, toggle_buttons)
display(grid, interact_out)
display(bar)